<a href="https://colab.research.google.com/github/Liorinio/Animal-Classification/blob/main/Animal_Classsification_Application_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install gradio
!pip install gTTS

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.2/54.2 MB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.1/323.1 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 32.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 kB 4.2 MB/s eta 0:00:00
  Attempting uninstall: click
    Found existing installation: click 8.2.0
    Uninstalling click-8.2.0:
      Successfully uninstalled click-8.2.0


In [2]:
from gtts import gTTS
import IPython.display as display
import gradio as gr
import pandas as pd
from google.colab import drive
import time
import joblib

from sklearn.preprocessing import LabelEncoder
import torch
import torch.nn as nn
import timm
from torchvision import models
from torchvision import transforms
from PIL import Image
import io

In [3]:
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [4]:
def txt_to_dataframe(file_path):
    with open(file_path, "r", encoding="utf-8") as file:
        paragraphs = [para.strip() for para in file.read().strip().split("\n\n") if para.strip()]

    df = pd.DataFrame({"Paragraph": paragraphs}, dtype=str)  # Ensure all values are strings
    return df

def say_text(text, lang="iw"):
    tts = gTTS(text=text, lang=lang)  # Convert text to speech
    tts.save("speech.mp3")  # Save the audio file
    display.display(display.Audio("speech.mp3", autoplay=True))  # Play the audio

In [5]:
df = txt_to_dataframe('/content/drive/MyDrive/Datasets/Animals brief descriptions.txt')
df.head()

,Paragraph
0,Antelopes are graceful herbivores found primar...
1,Badgers are burrowing mammals recognized by th...
2,Bats are the only mammals capable of sustained...
3,Bears are large mammals found across various c...
4,Bees are flying insects closely related to was...


In [6]:
df.tail()

,Paragraph
90,Wolves are social carnivores known for their c...
91,"Wombats are sturdy, burrowing marsupials nativ..."
92,Woodpeckers are birds characterized by their s...
93,"Worms are elongated, soft-bodied invertebrates..."
94,Zebras are African equids known for their dist...


In [7]:
animals_names = ['antelope', 'badger', 'bat', 'bear', 'bee', 'beetle', 'bison', 'boar', 'butterfly', 'cat', 'caterpillar', 'chimpanzee', 'cockroach', 'common loon', 'cow', 'coyote', 'crab', 'crow', 'deer', 'dog', 'dolphin', 'donkey', 'dragonfly', 'duck', 'eagle', 'elephant', 'flamingo', 'fly', 'fox', 'giraffe', 'goat', 'goldfish', 'goose', 'gorilla', 'grasshopper', 'hamster', 'hare', 'hedgehog', 'hippopotamus', 'hornbill', 'horse', 'hummingbird', 'hyena', 'jellyfish', 'kangaroo', 'koala', 'ladybugs', 'leopard', 'lion', 'lizard', 'lobster', 'mosquito', 'moth', 'mouse', 'octopus', 'okapi', 'orangutan', 'ostrich', 'otter', 'owl', 'ox', 'oyster', 'panda', 'parrot', 'pelican', 'penguin', 'pig', 'pigeon', 'porcupine', 'possum', 'raccoon', 'rat', 'red panda', 'reindeer', 'rhinoceros', 'sandpiper', 'seahorse', 'seal', 'shark', 'sheep', 'snake', 'sparrow', 'squid', 'squirrel', 'starfish', 'swan', 'tiger', 'turkey', 'turtle', 'whale', 'wolf', 'wombat', 'woodpecker', 'worm', 'zebra']
df_animals_names = pd.DataFrame(animals_names, columns=["animals_names"])
df_animals_names.head()

,animals_names
0,antelope
1,badger
2,bat
3,bear
4,bee


In [8]:
df_animals_names.shape

(95, 1)

In [9]:
df_animals_classes  = pd.concat([df_animals_names, df], axis=1)
df_animals_classes.head(10)

,animals_names,Paragraph
0,antelope,Antelopes are graceful herbivores found primar...
1,badger,Badgers are burrowing mammals recognized by th...
2,bat,Bats are the only mammals capable of sustained...
3,bear,Bears are large mammals found across various c...
4,bee,Bees are flying insects closely related to was...
5,beetle,Beetles constitute the largest order of insect...
6,bison,"Bison are large, grazing mammals native to Nor..."
7,boar,"Boars are wild relatives of domestic pigs, kno..."
8,butterfly,Butterflies are insects celebrated for their c...
9,cat,"Cats are small, carnivorous mammals that have ..."


In [10]:
'''
import time
from gtts import gTTS
import IPython.display as display

def say_brief_description_from_image(img):
    # Get the predicted label and confidence from the classifier
    prediction = get_animal_label_from_prediction(img)

    # Extract the animal name from the prediction dictionary
    animal_name = list(prediction.keys())[0]

    # Get the description from the dataframe
    text = df_animals_classes.loc[df_animals_classes['animals_names'].str.lower() == animal_name.lower(), 'Paragraph'].iloc[0]

    # Generate the speech using gTTS
    tts = gTTS(text=text, lang="en")
    tts.save("brief_description.mp3")

    # Pause before playing
    time.sleep(2)

    # Play the audio
    audio = display.Audio("brief_description.mp3", autoplay=True)
    display.display(audio)
    '''

'\nimport time\nfrom gtts import gTTS\nimport IPython.display as display\n\ndef say_brief_description_from_image(img):\n    # Get the predicted label and confidence from the classifier\n    prediction = get_animal_label_from_prediction(img)\n\n    # Extract the animal name from the prediction dictionary\n    animal_name = list(prediction.keys())[0]\n\n    # Get the description from the dataframe\n    text = df_animals_classes.loc[df_animals_classes[\'animals_names\'].str.lower() == animal_name.lower(), \'Paragraph\'].iloc[0]\n\n    # Generate the speech using gTTS\n    tts = gTTS(text=text, lang="en")\n    tts.save("brief_description.mp3")\n\n    # Pause before playing\n    time.sleep(2)\n\n    # Play the audio\n    audio = display.Audio("brief_description.mp3", autoplay=True)\n    display.display(audio)\n    '

In [11]:
!git clone https://huggingface.co/spaces/Liorinio/Animal-Classification

Cloning into 'Animal-Classification'...
fatal: could not read Username for 'https://huggingface.co': No such device or address


In [12]:
# ✅ Define Custom CNN + FFNN Classifier
class CustomCNNFFNN(nn.Module):
    def __init__(self, num_classes):
        super(CustomCNNFFNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.pool1 = nn.MaxPool2d(2, 2)

        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.pool2 = nn.MaxPool2d(2, 2)

        self.dropout = nn.Dropout(0.5)

        # Calculate flattened_size dynamically based on input size (224x224)
        self.flattened_size = 64 * 56 * 56  # After conv layers on 224x224 input

        self.fc1 = nn.Linear(self.flattened_size, 1024)
        self.fc2 = nn.Linear(1024, 512)
        self.fc3 = nn.Linear(512, 256)
        self.fc4 = nn.Linear(256, 128)
        self.fc5 = nn.Linear(128, num_classes)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.pool1(self.relu(self.bn1(self.conv1(x)))) # Pass through conv1, bn1, relu, and pool1
        x = self.pool2(self.relu(self.bn2(self.conv2(x)))) # Pass through conv2, bn2, relu, and pool2
        x = x.view(x.size(0), -1) # Flatten the output for the fully connected layers
        x = self.dropout(x)
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.relu(self.fc3(x))
        x = self.relu(self.fc4(x))
        x = self.fc5(x)
        return x


# Load the saved LabelEncoder
label_encoder = joblib.load("/content/drive/MyDrive/LabelEncoder/labelencoder.pkl")
# Retrieve the class names from the loaded encoder
class_names = list(label_encoder.classes_)

all_labels = class_names
label_encoder.fit(all_labels)


# ✅ Load Pretrained ResNet18
resnet = models.resnet18(pretrained=True)

# ✅ Freeze Feature Extractor
for param in resnet.parameters():
    param.requires_grad = False

# ✅ Replace Classifier with a New Linear Layer
num_classes = len(label_encoder.classes_)
in_features = resnet.fc.in_features
resnet.fc = nn.Linear(in_features, num_classes)  # ResNet18 uses 'fc' for the classifier

# ✅ Unfreeze Classifier
for param in resnet.fc.parameters():
    param.requires_grad = True

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 185MB/s]


In [13]:
print(class_names)

[np.str_('0'), np.str_('1'), np.str_('10'), np.str_('11'), np.str_('12'), np.str_('13'), np.str_('14'), np.str_('15'), np.str_('16'), np.str_('17'), np.str_('18'), np.str_('19'), np.str_('2'), np.str_('20'), np.str_('21'), np.str_('22'), np.str_('23'), np.str_('24'), np.str_('25'), np.str_('26'), np.str_('27'), np.str_('28'), np.str_('29'), np.str_('3'), np.str_('30'), np.str_('31'), np.str_('32'), np.str_('33'), np.str_('34'), np.str_('35'), np.str_('36'), np.str_('37'), np.str_('38'), np.str_('39'), np.str_('4'), np.str_('40'), np.str_('41'), np.str_('42'), np.str_('43'), np.str_('44'), np.str_('45'), np.str_('46'), np.str_('47'), np.str_('48'), np.str_('49'), np.str_('5'), np.str_('50'), np.str_('51'), np.str_('52'), np.str_('53'), np.str_('54'), np.str_('55'), np.str_('56'), np.str_('57'), np.str_('58'), np.str_('59'), np.str_('6'), np.str_('60'), np.str_('61'), np.str_('62'), np.str_('63'), np.str_('64'), np.str_('65'), np.str_('66'), np.str_('67'), np.str_('68'), np.str_('69'), n

In [14]:
# ✅ Define device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ✅ Define Image Transform (No augmentation for inference)
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

# ✅ Load Pretrained Model with Custom Classifier
def load_model(model_path):
    # Load the model with the number of classes in the saved model (95)
    model = timm.create_model("resnet18", pretrained=False, num_classes=95)
    model.load_state_dict(torch.load(model_path, map_location=device))

    model.to(device)
    model.eval()
    return model

# ✅ Load model (make sure the model file is uploaded first)
model = load_model('/content/drive/MyDrive/savedSecondModelProjectAnimals.pth')  # Replace with your file name

In [15]:
def classify_image(img):
    # Resize and convert to RGB if needed
    img = img.resize((224, 224)).convert("RGB")

    # Apply transformations and move to device (CPU/GPU)
    input_tensor = transform(img).unsqueeze(0).to(device)

    # Disable gradient calculation (inference mode)
    with torch.no_grad():
        outputs = model(input_tensor)
        probabilities = torch.softmax(outputs, dim=1)
        confidence, predicted_class = torch.max(probabilities, 1)

    # Return dictionary: {class_name: confidence_score}
    predicted_label = class_names[predicted_class.item()]
    return {predicted_label: float(confidence.item())}

In [16]:

import json

with open('/content/drive/MyDrive/LabelDict.json', 'r') as f:
        label_dict = json.load(f)  # Define label_dict here

def get_animal_label_from_prediction(img):
    predicted_dict = classify_image(img)  # Use classify_image internally
    predicted_key = int(list(predicted_dict.keys())[0])  # Convert string key to int


    # The following line was incorrectly indented
    inv_label_dict = {v: k for k, v in label_dict.items()}
    predicted_label = inv_label_dict.get(predicted_key, "Unknown")
    confidence_score = predicted_dict[str(predicted_key)]
    return {predicted_label: confidence_score}


In [17]:

from gtts import gTTS
import os

def classify_and_speak(img):
    # Step 1: Predict animal name and confidence
    prediction = get_animal_label_from_prediction(img)
    animal_name = list(prediction.keys())[0]

    # Step 2: Get description
    text = df_animals_classes.loc[
        df_animals_classes['animals_names'].str.lower() == animal_name.lower(), 'Paragraph'
    ].iloc[0]

    # Step 3: Generate audio
    tts = gTTS(text=text, lang="en")
    audio_path = "/tmp/brief_description.mp3"
    tts.save(audio_path)

    # Step 4: Return prediction and audio file
    return prediction, audio_path


In [18]:
import gradio as gr

interface = gr.Interface(
    fn=classify_and_speak,
    inputs=gr.Image(type="pil", label="Upload Animal Image", sources=["upload"]),
    outputs=[
        gr.Label(num_top_classes=1, label="Predicted Animal"),
        gr.Audio(label="Click to Hear Description", autoplay=True)
    ],
    title="🐾 Animal Image Classifier",
    description="Upload an animal image to classify it and hear a spoken description.",
    allow_flagging="never"
)

interface.launch()

/usr/local/lib/python3.11/dist-packages/gradio/interface.py:416: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated.Use `flagging_mode` instead.
  warnings.warn(


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://db11f20a16d1b4273e.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
